# Get Data

In [72]:
import nltk
nltk.download('punkt')
import math

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/BakerStreetBakery/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [101]:
fs = open('./data/1.txt', 'r')
text = fs.read().decode('utf-8')

In [102]:
text[:100]

u'THE ADVENTURES OF SHERLOCK HOLMES\n\nby\n\nSIR ARTHUR CONAN DOYLE\n\n\n\n   I. A Scandal in Bohemia\n  II. Th'

In [49]:
tokens = nltk.word_tokenize(text)
words = [word.lower() for word in tokens if word.isalpha()]

print 'tokens length', len(tokens)
print 'words length', len(words)

tokens length 125682
words length 102881


In [50]:
words[:100]

[u'the',
 u'adventures',
 u'of',
 u'sherlock',
 u'holmes',
 u'by',
 u'sir',
 u'arthur',
 u'conan',
 u'doyle',
 u'i',
 u'a',
 u'scandal',
 u'in',
 u'bohemia',
 u'ii',
 u'the',
 u'league',
 u'iii',
 u'a',
 u'case',
 u'of',
 u'identity',
 u'iv',
 u'the',
 u'boscombe',
 u'valley',
 u'mystery',
 u'the',
 u'five',
 u'orange',
 u'pips',
 u'vi',
 u'the',
 u'man',
 u'with',
 u'the',
 u'twisted',
 u'lip',
 u'vii',
 u'the',
 u'adventure',
 u'of',
 u'the',
 u'blue',
 u'carbuncle',
 u'viii',
 u'the',
 u'adventure',
 u'of',
 u'the',
 u'speckled',
 u'band',
 u'ix',
 u'the',
 u'adventure',
 u'of',
 u'the',
 u'engineer',
 u'thumb',
 u'x',
 u'the',
 u'adventure',
 u'of',
 u'the',
 u'noble',
 u'bachelor',
 u'xi',
 u'the',
 u'adventure',
 u'of',
 u'the',
 u'beryl',
 u'coronet',
 u'xii',
 u'the',
 u'adventure',
 u'of',
 u'the',
 u'copper',
 u'beeches',
 u'adventure',
 u'i',
 u'a',
 u'scandal',
 u'in',
 u'bohemia',
 u'i',
 u'to',
 u'sherlock',
 u'holmes',
 u'she',
 u'is',
 u'always',
 u'the',
 u'woman',
 u'

# Raw Frequency

In [54]:
min_threshold_tau = 5
words_len = len(words)

In [57]:
f = {}
index = {}
for i in range(words_len):
    word = words[i]
    if word not in index.keys():
        index[word] = [i]
    else:
        index[word].append(i)


In [61]:
print index.keys()[:10]
print index[index.keys()[1]]

[u'writings', u'yellow', u'four', u'woods', u'hanging', u'pardon', u'granting', u'eligible', u'originality', u'lord']
[16100, 29961, 41106, 43059, 55336, 62004, 65537, 66533, 71653, 73134, 77788, 93519, 96441]


In [65]:
while len(index.keys()) > 0:
    index_p = {}
    for j in index.keys():
        if len(index[j]) > min_threshold_tau:
            f[j] = len(index[j])
            for k in index[j]:
                if k+1 < words_len:
                    new_phrase = j + u' ' + words[k + 1]
                    if new_phrase not in index_p.keys():
                        index_p[new_phrase] = [k + 1]
                    else:
                        index_p[new_phrase].append(k + 1)
    index = index_p
    print index.keys()[:10]

[u'dad is', u'city the', u'it absolutely', u'bird then', u'dashed down', u'expect the', u'your whole', u'stairs it', u'now when', u'all quarters']
[u'that we were', u'down the sheet', u'you are peterson', u'anyone else yes', u'he took it', u'was no more', u'to a clever', u'me with my', u'would be for', u'said i laughing']
[u'would not have his', u'he was in dreadful', u'i believe that the', u'that it is still', u'as i have changed', u'more than once observed', u'sherlock holmes was as', u'rushed into the glade', u'if it were guilty', u'to say that a']
[u'i would not have his', u'one of the most determined', u'on the other hand if', u'to me to be a', u'one of the most absolute', u'i do not know is', u'i do not think that', u'seemed to me that i', u'the corner of the mantelpiece', u'one of the most lovely']
[u'i beg that you will draw', u'i have no doubt that you', u'from one to the other noting', u'from one to the other pausing', u'i have no doubt that he', u'i have no doubt that we', u

In [66]:
print len(f.keys())
print f.keys()[:10]

4581
[u'to me that', u'could not', u'to me and', u'the second', u'give him', u'until at', u'yellow', u'along the', u'four', u'while he']


# Quality Score

In [67]:
phrases = []
for w in f.keys():
    if len(w.split(u' ')) > 1:
        phrases.append(w)

In [70]:
print len(phrases)
print phrases[:10]

2887
[u'to me that', u'could not', u'to me and', u'the second', u'give him', u'until at', u'along the', u'while he', u'night the', u'the hands']


# Concordance

In [94]:
PMI = {}
PLK = {}

In [95]:
def prob_u(phrase, f):
    words = phrase.split(u' ')
    denom = 0
    for j in words:
        denom += f[word]
    return f[phrase]*1. / denom

In [96]:
def cal_PMI_PLK(phrase, f, pmi, plk):
    words = phrase.split(u' ')
    prob_whole = prob_u(phrase, f)
    if len(words) > 2:
        min_mutual_info = float('inf')
        best_u_left = None
        best_u_right = None
        for i in range(1, len(words)):
            u_left = u' '.join(words[:i])
            u_right = u' '.join(words[i:])
            info = math.log(prob_whole * 1. / (prob_u(u_left, f) * prob_u(u_right, f)))
            if info < min_mutual_info:
                min_mutual_info = info
                best_u_left = u_left
                best_u_right = u_right
    else:
        best_u_left = words[0]
        best_u_right = words[1]
    
    pmi[phrase] = math.log(prob_whole * 1. / (prob_u(best_u_left, f) * prob_u(best_u_right, f)))
    plk[phrase] = prob_whole * math.log(prob_whole * 1. / (prob_u(best_u_left, f) * prob_u(best_u_right, f)))


In [97]:
for p in phrases:
    cal_PMI_PLK(p, f, PMI, PLK)

In [98]:
print len(PMI.keys())
print PMI.keys()[0]
print PMI[PMI.keys()[0]]
print PLK[PLK.keys()[0]]

2887
to me that
-7.51906999331
-4.76207766243


# Informativeness

In [112]:
IDF_phrase = {}
IDF_word = {}


In [117]:
sentences = text.split('\n')
len(sentences)
for i in range(len(sentences)):
    sentences[i] = sentences[i].lower()
print sentences[0]

the adventures of sherlock holmes


In [118]:
def cal_word_IDF(docs, idf_word):
    total_length = len(docs)
    for w in words:
        count = 0
        for d in docs:
            if w in d:
                count += 1
        if count > 0:
            idf_word[w] = math.log(total_length * 1. / count)
        else:
            idf_word[w] = 0
        
        

In [ ]:
cal_word_IDF(sentences, IDF_word)

In [ ]:
print IDF_word.keys()[:5]
print IDF_word[IDF_word.keys()[0]]